In [2]:
__author__ = 'Wilson Apaza'

from PyQt5 import QtWidgets, uic
from PyQt5.QtWidgets import QMessageBox
from PyQt5.QtCore import QUrl
from PyQt5.QtGui import QImage
from PyQt5.QtGui import QPixmap
from PyQt5 import QtCore						#importar bibliotecas pyqt5
#from scipy.ndimage import imread				# Lectura de las imágenes
from PyQt5.QtCore import QTimer,Qt 
from PyQt5 import QtGui							
from tkinter import filedialog					#Módulo de exportación de archivosq
from tkinter import * 
import tkinter as tk
from matplotlib import pyplot as plt 			#Espectador de gestos
from matplotlib.widgets import Button
import sys										#Archivo añadido extensión pyqt
import os										#Eleccion y Eliminación de archivos
import cv2										#Operaciones de la cámara
import numpy as np 								#Proceso de las imagenes
import qimage2ndarray 							#convierte imágenes en matriz
import win32api
import winGuiAuto			
import win32gui
import win32con									#Eliminar la ventana del título cv2 y siempre en la parte superior
import keyboard									#presionar teclas para accionar programa 
import pyttsx3									#Asistente virtual
import shutil									#Directorios
index = 0										#índice utilizado para el visor de gestos
engine = pyttsx3.init()							#inicio de asistente de audio 

def nothing(x):
	pass

image_x, image_y = 64,64						#resolucion de imagen 

from tensorflow.keras.models import load_model
classifier = load_model('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/CNN_checkpoint.h5')			#cargamos modelo entrenador 

def fileSearch():
	"""Busca cada archivo que termina con .png en el directorio GestosCreados para que el gesto personalizado se 
    pueda pasar a la función predictor ()"""
	fileEntry=[]
	for file in os.listdir("C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados"):
	    if file.endswith(".png"):
	    	fileEntry.append(file)
	return fileEntry							

def load_images_from_folder(folder):
    """Busca cada imagen en un directorio específico"""
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

def toggle_imagesfwd(event):
	"""Muestra las siguientes imágenes actuar que se guardaron"""
	img=load_images_from_folder('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/TempGest/')
	global index

	index += 1

	try:
		if index < len(img):
			plt.axes()
			plt.imshow(img[index])
			plt.draw()
	except:
		pass

def toggle_imagesrev(event):
	"""muestra imágenes anteriores para ser mostradas"""
	img=load_images_from_folder('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/TempGest/')
	global index

	index -= 1

	try:
		if index < len(img) and index>=0:
			plt.axes()
			plt.imshow(img[index])
			plt.draw()
	except:
		pass

def openimg():
	"""Muestra resultado de gestos predefinidos en la ventana"""
	cv2.namedWindow("Image", cv2.WINDOW_NORMAL )
	image = cv2.imread('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/template.png')
	cv2.imshow("Image",image)
	cv2.setWindowProperty("Image",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
	cv2.resizeWindow("Image",298,430)
	cv2.moveWindow("Image", 1052,214)

def removeFile():
	"""Elimina el directorio temp.txt y tempgest si se presiona cualquier botón de parada o se cierra la aplicación"""
	try:
		os.remove("temp.txt")
	except:
		pass
	try:
		shutil.rmtree("C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/TempGest")
	except:
		pass

def clearfunc(cam):
	"""Apaga la cámara abierta y reinicia funcion remove"""
	cam.release()
	cv2.destroyAllWindows()
	removeFile()

def clearfunc2(cam):
	"""Apaga la camara encendida"""
	cam.release()
	cv2.destroyAllWindows()

def saveBuff(self,cam,finalBuffer):
	"""Guardamos el archivo como temp.txt si se presiona el botón Guardar en la formación de la oración a través 
    de la interfaz gráfica de usuario"""
	cam.release()
	cv2.destroyAllWindows()
	if(len(finalBuffer)>=1):
		f=open("temp.txt","w")
		for i in finalBuffer:
			f.write(i)
		f.close()

def capture_images(self,cam,saveimg,mask):
	"""Guarda las imágenes para gestos personalizados si se presiona el botón en la generación de gestos
    personalizados a través de la interfaz gráfica de usuario"""
	cam.release()
	cv2.destroyAllWindows()
	if not os.path.exists('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados'):
		os.mkdir('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados')

	gesname=saveimg[-1]
	if(len(gesname)>=1):
		img_name = "C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados/"+"{}.png".format(str(gesname))
		save_img = cv2.resize(mask, (image_x, image_y))
		cv2.imwrite(img_name, save_img)


def controlTimer(self):
	# if timer is stopped
	self.timer.isActive()
	# create video capture
	self.cam = cv2.VideoCapture(0)
	# start timer
	self.timer.start(20)
            

def predictor():
	""" Dependiendo del modelo cargado y la predicción guardada de la gestión personalizada se realiza verificando la matriz"""
	import numpy as np
	from tensorflow.keras.preprocessing import image
	test_image = image.load_img('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/1.png', target_size=(64, 64))
	test_image = image.img_to_array(test_image)
	test_image = np.expand_dims(test_image, axis = 0)
	result = classifier.predict(test_image)
	gesname=''
	fileEntry=fileSearch()
	for i in range(len(fileEntry)):
		image_to_compare = cv2.imread("C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados/"+fileEntry[i])
		original = cv2.imread("C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/1.png")
		sift = cv2.xfeatures2d.SIFT_create()
		kp_1, desc_1 = sift.detectAndCompute(original, None)
		kp_2, desc_2 = sift.detectAndCompute(image_to_compare, None)

		index_params = dict(algorithm=0, trees=5)
		search_params = dict()
		flann = cv2.FlannBasedMatcher(index_params, search_params)

		matches = flann.knnMatch(desc_1, desc_2, k=2)

		good_points = []
		ratio = 0.6
		for m, n in matches:
		      if m.distance < ratio*n.distance:
		             good_points.append(m)
		if(abs(len(good_points)+len(matches))>20):		    #gestos personalizados menos que 20 
			gesname=fileEntry[i]
			gesname=gesname.replace('.png','')
			if(gesname=='sp'):								#Capturador de imagenes personalizadas
				gesname=' '
			return gesname

	if result[0][0] == 1:
		  return 'A'
	elif result[0][1] == 1:
		  return 'B'
	elif result[0][2] == 1:
		  return 'C'
	elif result[0][3] == 1:
		  return 'D'
	elif result[0][4] == 1:
		  return 'E'
	elif result[0][5] == 1:
		  return 'F'
	elif result[0][6] == 1:
		  return 'G'
	elif result[0][7] == 1:
		  return 'H'
	elif result[0][8] == 1:
		  return 'I'
	elif result[0][9] == 1:
		  return 'J'
	elif result[0][10] == 1:
		  return 'K'
	elif result[0][11] == 1:
		  return 'L'
	elif result[0][12] == 1:
		  return 'M'
	elif result[0][13] == 1:
		  return 'N'
	elif result[0][14] == 1:
		  return 'O'
	elif result[0][15] == 1:
		  return 'P'
	elif result[0][16] == 1:
		  return 'Q'
	elif result[0][17] == 1:
		  return 'R'
	elif result[0][18] == 1:
		  return 'S'
	elif result[0][19] == 1:
		  return 'T'
	elif result[0][20] == 1:
		  return 'U'
	elif result[0][21] == 1:
		  return 'V'
	elif result[0][22] == 1:
		  return 'W'
	elif result[0][23] == 1:
		  return 'X'
	elif result[0][24] == 1:
		  return 'Y'
	elif result[0][25] == 1:
		  return 'Z'
			   

def checkFile():
	"""Recuperar el contenido de temp.txt para el módulo de exportación y se interpreta mediante el asisente virtual"""
	checkfile=os.path.isfile('temp.txt')
	if(checkfile==True):
		fr=open("temp.txt","r")
		content=fr.read()
		fr.close()
	else:
		content="TRADUCTOR DE SEÑAS UNIVERSIDAD TECNOLOGICA DEL PERU"
	return content


class Dashboard(QtWidgets.QMainWindow):
	def __init__(self):
		super(Dashboard, self).__init__()
		self.setWindowFlags(QtCore.Qt.WindowMinimizeButtonHint)
		cap = cv2.VideoCapture('videoinicio.mp4') 
		   
		# Leer hasta que se complete la activacion de la camra 
		while(cap.isOpened()):
			ret, frame = cap.read()
			if ret == True:
		# Mascara para capturar los frame 
				ret, frame = cap.read() 
				cv2.namedWindow("mask", cv2.WINDOW_NORMAL)
				cv2.imshow("mask", frame)
				cv2.setWindowProperty("mask",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
				cv2.resizeWindow("mask",720,400)
				cv2.moveWindow("mask", 320,220)
				
				if cv2.waitKey(25) & 0xFF == ord('q'):
					break

			else:
				break
		 
		# Cuando todas las funciones esten declaradas se analiza 
		cap.release()
		   
		# Cierra los frames 
		cv2.destroyAllWindows()
        
		self.setWindowIcon(QtGui.QIcon('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/icons/windowLogo.png'))
		self.title = 'Traductor de Lenguaje de Señas UTP V2.00'
		uic.loadUi('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/UI_Files/dash.UI', self)
		self.setWindowTitle(self.title)
		self.timer = QTimer()
		self.create.clicked.connect(self.createGest)
		self.exp2.clicked.connect(self.exportFile)
		self.scan_sen.clicked.connect(self.scanSent)
		if(self.scan_sinlge.clicked.connect(self.scanSingle)==True):
			self.timer.timeout.connect(self.scanSingle)
		self.create.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sen.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sinlge.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.exp2.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.exit_button.clicked.connect(self.quitApplication)
		self._layout = self.layout()
		self.label_3 = QtWidgets.QLabel()
		movie = QtGui.QMovie("C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/dashAnimation.gif")
		self.label_3.setMovie(movie)
		self.label_3.setGeometry(0,160,780,441)
		movie.start()
		self._layout.addWidget(self.label_3)
		self.setObjectName('Message_Window')
		
	def quitApplication(self):
		"""Cerrar sesion de la ventana GUI junto con la eliminación de archivos"""
		userReply = QMessageBox.question(self, 'Salir del Programa', "¿Estás segura de que quieres salir del programa?", QMessageBox.Yes | QMessageBox.No, QMessageBox.No)
		if userReply == QMessageBox.Yes:
			removeFile()
			keyboard.press_and_release('alt+F4')

	def createGest(self):
		""" Módulo de generación de gestos personalizados"""
		try:
			clearfunc(self.cam)
		except:
			pass
		gesname=""
		uic.loadUi('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/UI_Files/create_gest.ui', self)
		self.setWindowTitle(self.title)
		self.create.clicked.connect(self.createGest)
		self.exp2.clicked.connect(self.exportFile)
		if(self.scan_sen.clicked.connect(self.scanSent)):
			controlTimer(self)
		self.scan_sinlge.clicked.connect(self.scanSingle)
		self.linkButton.clicked.connect(openimg)
		self.create.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sen.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sinlge.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.exp2.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.pushButton_2.clicked.connect(lambda:clearfunc(self.cam))
		try:
			self.exit_button.clicked.connect(lambda:clearfunc(self.cam))
		except:
			pass
		self.exit_button.clicked.connect(self.quitApplication)
		self.plainTextEdit.setPlaceholderText("Nombre de tu imagen") 
		img_text = ''
		saveimg=[]
		while True:
			ret, frame = self.cam.read()
			frame = cv2.flip(frame,1)
			try:
				frame=cv2.resize(frame,(321,270))
				frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
				img2 = cv2.rectangle(frame, (150,50),(300,200), (0,255,0), thickness=2, lineType=8, shift=0)
			except:
				keyboard.press_and_release('esc')

			height2, width2, channel2 = img2.shape
			step2 = channel2 * width2
        	# crear QImage a partir de la imagen
			qImg2 = QImage(img2.data, width2, height2, step2, QImage.Format_RGB888)
        	# mostrar imagen en img_label
			try:
				self.label_3.setPixmap(QPixmap.fromImage(qImg2))
				slider2=self.trackbar.value()
			except:
				pass

			lower_blue = np.array([0, 0, 0])
			upper_blue = np.array([179, 255, slider2])
			imcrop = img2[52:198, 152:298]
			hsv = cv2.cvtColor(imcrop, cv2.COLOR_BGR2HSV)
			mask = cv2.inRange(hsv, lower_blue, upper_blue)
			         
			cv2.namedWindow("mask", cv2.WINDOW_NORMAL )
			cv2.imshow("mask", mask)
			cv2.setWindowProperty("mask",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
			cv2.resizeWindow("mask",170,160)
			cv2.moveWindow("mask", 766,271)

			hwnd = winGuiAuto.findTopWindow("mask")
			win32gui.SetWindowPos(hwnd, win32con.HWND_TOP, 0,0,0,0,win32con.SWP_NOMOVE | win32con.SWP_NOSIZE | win32con.SWP_NOACTIVATE)


			try:
				ges_name = self.plainTextEdit.toPlainText()
			except:
				pass
			if(len(ges_name)>=1):
				saveimg.append(ges_name)
			else:
				saveimg.append(ges_name)
				ges_name=''

			try:
				self.pushButton.clicked.connect(lambda:capture_images(self,self.cam,saveimg,mask))
			except:
				pass			
			
			gesname=saveimg[-1]

			if keyboard.is_pressed('shift+s'):
				if not os.path.exists('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados'):
					os.mkdir('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados')
				if(len(gesname)>=1):
					img_name = "C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados/"+"{}.png".format(str(gesname))
					save_img = cv2.resize(mask, (image_x, image_y))
					cv2.imwrite(img_name, save_img)
				break

			if cv2.waitKey(1) == 27:
				break
		
		self.cam.release()
		cv2.destroyAllWindows()

		if os.path.exists("C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/GestosCreados/"+str(gesname)+".png"):
			QtWidgets.QMessageBox.about(self, " UTP :)  ", "Imagen Guardada con Exito")

	def exportFile(self):
		"""Funcion exportar y visualizar gestos creados"""
		try:
			clearfunc2(self.cam)
		except:
			pass
		uic.loadUi('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/UI_Files/export.UI', self)
		self.setWindowTitle(self.title)
		self.create.clicked.connect(self.createGest)
		self.exp2.clicked.connect(self.exportFile)
		self.scan_sen.clicked.connect(self.scanSent)
		self.scan_sinlge.clicked.connect(self.scanSingle)
		self.create.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sen.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sinlge.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.exp2.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.exit_button.clicked.connect(self.quitApplication)
		content=checkFile()
		self.textBrowser_98.setText("		 "+content)
		engine.say(str(content).lower())
		try:
			engine.runAndWait()
		except:
			pass
		if(content=="Archivo no encontrado"):
			self.pushButton_2.setEnabled(False)
			self.pushButton_3.setEnabled(False)
		else:
			self.pushButton_2.clicked.connect(self.on_click)
			try:
				self.pushButton_3.clicked.connect(self.gestureViewer)
			except:
				pass

	def on_click(self):
		"""Abre la ventana de tkinter para guardar el archivo en la ubicación deseada"""
		content=checkFile()
		root=Tk()
		root.withdraw()
		root.filename =  filedialog.asksaveasfilename(initialdir = "/",title = "Select file",filetypes = (("Text files","*.txt"),("all files","*.*")))
		name=root.filename
		#fr.close()
		fw=open(name+".txt","w")
		if(content=='Bienvenido Alumno'):
			content=" "
		fw.write(content)
		try:
			os.remove("temp.txt")
			shutil.rmtree("C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/TempGest")
		except:
			QtWidgets.QMessageBox.about(self, "Informacion", "Nada exportado")
		fw.close()
		root.destroy()

		if not os.path.exists('temp.txt'):
			if os.path.exists('.txt'):
				os.remove('.txt')
			else:
				QtWidgets.QMessageBox.about(self, "Informacion", "Archivo guardado exitosamente!")
				self.textBrowser_98.setText("		 ")
	
	def gestureViewer(self):
		"""Vemos el gesto en Matplotlib"""
		try:
			img=load_images_from_folder('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/TempGest/')
			plt.imshow(img[index])
		except:
			plt.text(0.5, 0.5, 'No new Gesture Available', horizontalalignment='center',verticalalignment='center')
		axcut = plt.axes([0.9, 0.0, 0.1, 0.075])
		axcut1 = plt.axes([0.0, 0.0, 0.1, 0.075])
		bcut = Button(axcut, 'Next', color='dodgerblue', hovercolor='lightgreen')
		bcut1 = Button(axcut1, 'Previous', color='dodgerblue', hovercolor='lightgreen')

		#plt.connect('button_press_event', toggle_imagesfwd)
		bcut.on_clicked(toggle_imagesfwd)
		bcut1.on_clicked(toggle_imagesrev)
		plt.show()
		axcut._button = bcut 		#creamos la referencia del elemento
		axcut1._button1 = bcut1
	#buttonaxe._button = bcut


	def scanSent(self):
		"""módulo de formación de oraciones """
		try:
			clearfunc(self.cam)
		except:
			pass
		uic.loadUi('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/UI_Files/scan_sent.ui', self)
		self.setWindowTitle(self.title)
		self.create.clicked.connect(self.createGest)
		self.exp2.clicked.connect(self.exportFile)
		if(self.scan_sen.clicked.connect(self.scanSent)):
			controlTimer(self)
		self.scan_sinlge.clicked.connect(self.scanSingle)	
		try:
			self.pushButton_2.clicked.connect(lambda:clearfunc(self.cam))
		except:
			pass
		self.linkButton.clicked.connect(openimg)
		self.create.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sen.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sinlge.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.exp2.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		try:
			self.exit_button.clicked.connect(lambda:clearfunc(self.cam))
		except:
			pass
		self.exit_button.clicked.connect(self.quitApplication)
		img_text = ''
		append_text=''
		new_text=''
		finalBuffer=[]
		counts=0
		while True:
			ret, frame =self.cam.read()
			frame = cv2.flip(frame,1)
			try:
				frame=cv2.resize(frame,(331,310))
			
			
				frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
				img = cv2.rectangle(frame, (150,50),(300,200), (0,255,0), thickness=2, lineType=8, shift=0)
			except:
				keyboard.press_and_release('esc')
				keyboard.press_and_release('esc')

			height, width, channel = img.shape
			step = channel * width
        	# Creamos la Qimage
			qImg = QImage(img.data, width, height, step, QImage.Format_RGB888)
       		# mostrar imagen en img_label
			try:
				self.label_3.setPixmap(QPixmap.fromImage(qImg))
				slider=self.trackbar.value()
			except:
				pass
				
			lower_blue = np.array([0, 0, 0])
			upper_blue = np.array([179, 255, slider])
			imcrop = img[52:198, 152:298]
			hsv = cv2.cvtColor(imcrop, cv2.COLOR_BGR2HSV)
			mask1 = cv2.inRange(hsv, lower_blue, upper_blue)
			
			
			cv2.namedWindow("mask", cv2.WINDOW_NORMAL )
			cv2.imshow("mask", mask1)
			cv2.setWindowProperty("mask",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
			cv2.resizeWindow("mask",118,108)
			cv2.moveWindow("mask", 905,271)

			hwnd = winGuiAuto.findTopWindow("mask")
			win32gui.SetWindowPos(hwnd, win32con.HWND_TOP, 0,0,0,0,win32con.SWP_NOMOVE | win32con.SWP_NOSIZE | win32con.SWP_NOACTIVATE)

			try:
				self.textBrowser.setText("\n         "+str(img_text))
			except:
				pass
			img_name = "C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/1.png"
			save_img = cv2.resize(mask1, (image_x, image_y))
			cv2.imwrite(img_name, save_img)
			img_text=predictor()
			if cv2.waitKey(1) == ord('c'):
				try:
					counts+=1
					append_text+=img_text
					new_text+=img_text
					if not os.path.exists('./TempGest'):
						os.mkdir('C:/Users/PC/Desktop/PROYECTO2/wilson/Source CodeTempGest')
					img_names = "C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/TempGest/"+"{}{}.png".format(str(counts),str(img_text))
					save_imgs = cv2.resize(mask1, (image_x, image_y))
					cv2.imwrite(img_names, save_imgs)
					self.textBrowser_4.setText(new_text)
				except:
					append_text+=''
				
				if(len(append_text)>1):
					finalBuffer.append(append_text)
					append_text=''
				else:
					finalBuffer.append(append_text)
					append_text=''
					 
			try:
				self.pushButton.clicked.connect(lambda:saveBuff(self,self.cam,finalBuffer))
			except:
				pass
			if cv2.waitKey(1) == 27:
				break

			if keyboard.is_pressed('shift+s'):
				if(len(finalBuffer)>=1):
					f=open("temp.txt","w")
					for i in finalBuffer:
						f.write(i)
					f.close()
				break

			
		self.cam.release()
		cv2.destroyAllWindows()

		if os.path.exists('temp.txt'):
			QtWidgets.QMessageBox.about(self, "Informacion UTP", "Guardado con éxito.Gracias UTP")
		try:
			self.textBrowser.setText("		 ")
		except:
			pass

	def scanSingle(self):
		"""Escanear gesto sub - menu"""
		try:
			clearfunc(self.cam)
		except:
			pass
		uic.loadUi('C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/UI_Files/scan_single.ui', self)
		self.setWindowTitle(self.title)
		self.create.clicked.connect(self.createGest)
		self.exp2.clicked.connect(self.exportFile)
		self.scan_sen.clicked.connect(self.scanSent)
		if(self.scan_sinlge.clicked.connect(self.scanSingle)):
			controlTimer(self)
		self.pushButton_2.clicked.connect(lambda:clearfunc(self.cam))
		self.linkButton.clicked.connect(openimg)
		self.create.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sen.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.scan_sinlge.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		self.exp2.setCursor(QtGui.QCursor(QtCore.Qt.PointingHandCursor))
		try:
			self.exit_button.clicked.connect(lambda:clearfunc(self.cam))
		except:
			pass
		self.exit_button.clicked.connect(self.quitApplication)
		img_text = ''
		while True:
			ret, frame = self.cam.read()
			frame = cv2.flip(frame,1)
			try:
				frame=cv2.resize(frame,(321,270))
				frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
				img1 = cv2.rectangle(frame, (150,50),(300,200), (0,255,0), thickness=2, lineType=8, shift=0)
			except:
				keyboard.press_and_release('esc')

			height1, width1, channel1 = img1.shape
			step1 = channel1 * width1
        	# create QImage from image
			qImg1 = QImage(img1.data, width1, height1, step1, QImage.Format_RGB888)
        	# show image in img_label
			try:
				self.label_3.setPixmap(QPixmap.fromImage(qImg1))
				slider1=self.trackbar.value()
			except:
				pass

			lower_blue = np.array([0, 0, 0])
			upper_blue = np.array([179, 255, slider1])
			
			imcrop = img1[52:198, 152:298]
			hsv = cv2.cvtColor(imcrop, cv2.COLOR_BGR2HSV)
			mask = cv2.inRange(hsv, lower_blue, upper_blue)
			
			cv2.namedWindow("mask", cv2.WINDOW_NORMAL )
			cv2.imshow("mask", mask)
			cv2.setWindowProperty("mask",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
			cv2.resizeWindow("mask",118,108)
			cv2.moveWindow("mask", 894,271)

			hwnd = winGuiAuto.findTopWindow("mask")
			win32gui.SetWindowPos(hwnd, win32con.HWND_TOP, 0,0,0,0,win32con.SWP_NOMOVE | win32con.SWP_NOSIZE | win32con.SWP_NOACTIVATE)
			
			try:
				self.textBrowser.setText("\n\n\t"+str(img_text))
			except:
				pass

			img_name = "C:/Users/PC/Desktop/PROYECTO2/wilson/Source Code/1.png"
			save_img = cv2.resize(mask, (image_x, image_y))
			cv2.imwrite(img_name, save_img)
			img_text = predictor()
				
			if cv2.waitKey(1) == 27:
			    break

		self.cam.release()
		cv2.destroyAllWindows()

app = QtWidgets.QApplication([])
win = Dashboard()
win.show()
sys.exit(app.exec())

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x0000021FCC51ED10>.

In [3]:
from tensorflow.keras.models import load_model

# Guardar modelo
model.save('CNN_checkpoint.h5')

# Cargar modelo
loaded_model = load_model('CNN_checkpoint.h5')


NameError: name 'model' is not defined